In [10]:
# Импорт всех необходимых библиотек

import numpy as np
import torch
from torch.nn.functional import conv2d as libConv2d

In [25]:
class Conv2D:
    def __init__(self, input_data, kernel_size: tuple | int, bias: float | None = None,
                 stride: int = 1, padding: tuple[int, int] | int | str = (0, 0), dilation: int = 1):
        self.input_data_numpy = input_data[0, 0].numpy()
        self.input_data_torch = input_data
        self.bias = bias

        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size
        self.stride, self.dilation = stride, dilation

        self.padding = padding[0] if isinstance(padding, tuple) else \
                       (self.kernel_size[0] - 1 if padding == "same" and stride == 1 else 0)

        self.weight_tensor_torch = torch.randn(1, 1, *self.kernel_size)
        self.weight_tensor_numpy = self.weight_tensor_torch[0, 0].numpy()

    def conv2d(self):
        image_height, image_width = self.input_data_numpy.shape
        weight_height, weight_width = self.weight_tensor_numpy.shape
        H_out = int((image_height - self.dilation * (weight_height - 1) - 1 + 2 * self.padding) / self.stride) + 1
        W_out = int((image_width - self.dilation * (weight_width - 1) - 1 + 2 * self.padding) / self.stride) + 1

        self.input_data_numpy = np.pad(self.input_data_numpy, self.padding, mode='constant') if self.padding > 0 else self.input_data_numpy
        result = np.array([
            [np.sum(self.input_data_numpy[y * self.stride:y * self.stride + weight_height,
                   x * self.stride:x * self.stride + weight_width] * self.weight_tensor_numpy)
             for x in range(W_out)] for y in range(H_out)
        ])

        if self.bias:
            result += self.bias

        return result

    def torch_conv2d(self):
        return libConv2d(self.input_data_torch, self.weight_tensor_torch, self.bias, self.stride, self.padding,
                         self.dilation)

    def test(self, print_flag=False):
        my_conv2d = self.conv2d()
        torch_out = np.array(self.torch_conv2d())
        if print_flag:
            print(my_conv2d)
            print(torch_out[0, 0])
        print(np.allclose(my_conv2d, torch_out[0, 0]))

### Дальше идет проверка как раз-таки работоспособности моей функции сразу тест на сравнение со сверткой из торча, совпадают ли наши выходы и правильно ли все работает в общем:

#### ТЕСТЫ

In [26]:
# Tests
image = torch.randn(1, 1, 5, 5)
test1_out = Conv2D(image, kernel_size=1)
test1_out.test()

image = torch.randn(1, 1, 5, 5)
test2_out = Conv2D(image, kernel_size=1, padding='valid')
test2_out.test()

image = torch.randn(1, 1, 5, 5)
test3_out = Conv2D(image, kernel_size=1, padding='same')
test3_out.test()

image = torch.randn(1, 1, 5, 5)
test4_out = Conv2D(image, kernel_size=4, padding='same')
test4_out.test()

image = torch.randn(1, 1, 5, 5)
test5_out = Conv2D(image, kernel_size=1, dilation=3)
test5_out.test()

image = torch.randn(1, 1, 5, 5)
test6_out = Conv2D(image, kernel_size=1, stride=4)
test6_out.test()


True
True
True
True
True
True


### Как видно из результатов выше -  функция свертки работает отлично, результаты с оригинальной из торча совпадают с моей!